# General: configuration files

This example shows how you can use PyAEDT to export configuration files
and re-use them to import in a new project. A configuration file is
supported by these applications:

-   HFSS
-   2D Extractor and Q3D Extractor
-   Maxwell
-   Icepak (in AEDT)
-   Mechanical (in AEDT)

The following sections are covered:

-   Variables
-   Mesh operations (except Icepak)
-   Setup and optimetrics
-   Material properties
-   Object properties
-   Boundaries and excitations

When a boundary is attached to a face, the tool tries to match it with a
`FaceByPosition` on the same object name on the target design. If, for
any reason, this face position has changed or the object name in the
target design has changed, the boundary fails to apply.


# Perform required imports

Perform required imports from ansys.aedt.core.


In [ ]:
import os
import ansys.aedt.core

# Set AEDT version

Set AEDT version.


In [ ]:
aedt_version = "2024.2"

# Set non-graphical mode

Set non-graphical mode. You can set `non_graphical` either to `True` or
`False`.


In [ ]:
non_graphical = False

# Open project

Download the project, open it, and save it to the temporary folder.


In [ ]:
project_full_name = ansys.aedt.core.downloads.download_icepak(ansys.aedt.core.generate_unique_folder_name(folder_name="Graphic_Card"))

ipk = ansys.aedt.core.Icepak(project=project_full_name, version=aedt_version,
                    new_desktop=True, non_graphical=non_graphical)
ipk.autosave_disable()

# Create source blocks

Create a source block on the CPU and memories.


In [ ]:
ipk.create_source_block(object_name="CPU", input_power="25W")
ipk.create_source_block(object_name=["MEMORY1", "MEMORY1_1"], input_power="5W")

# Assign boundaries

Assign the opening and grille.


In [ ]:
region = ipk.modeler["Region"]
ipk.assign_openings(air_faces=region.bottom_face_x.id)
ipk.assign_grille(air_faces=region.top_face_x.id, free_area_ratio=0.8)

# Create setup

Create the setup. Properties can be set up from the `setup` object with
getters and setters. They don\'t have to perfectly match the property
syntax.


In [ ]:
setup1 = ipk.create_setup()
setup1["FlowRegime"] = "Turbulent"
setup1["Max Iterations"] = 5
setup1["Solver Type Pressure"] = "flex"
setup1["Solver Type Temperature"] = "flex"
ipk.save_project(r"C:\temp\Graphic_card.aedt")

# Export project to step file

Export the current project to the step file.


In [ ]:
filename = ipk.design_name
file_path = os.path.join(ipk.working_directory, filename + ".step")
ipk.export_3d_model(file_name=filename, file_path=ipk.working_directory, file_format=".step", assignment_to_export=[],
                    assignment_to_remove=[])

# Export configuration files

Export the configuration files. You can optionally disable the export
and import sections. Supported formats are json and toml files


In [ ]:
conf_file = ipk.configurations.export_config(os.path.join(ipk.working_directory, "config.toml"))
ipk.close_project()

# Create project

Create an Icepak project and import the step.


In [ ]:
app = ansys.aedt.core.Icepak(project="new_proj_Ipk")
app.modeler.import_3d_cad(file_path)

# Import and apply configuration file

Import and apply the configuration file. You can apply all or part of
the JSON file that you import using options in the `configurations`
object.


In [ ]:
out = app.configurations.import_config(conf_file)
app.configurations.results.global_import_success

# Close project

Close the project.


In [ ]:
app.release_desktop()